In [ ]:
from matplotlib import pyplot as plt 
import numpy as np 
import pandas as pd 
import rasterio 
import xarray as xr
import rioxarray 
from mesma.core import mesma, hard_classification

In [ ]:
def band_mask(wavelengths, mask_regions=[[1300,1500],[1800,2000]]):
    """
    Create a band mask for provided wavelength regions (default: water vapor regions at 1300-1500 nm and 1800-2000 nm.
    
    :param wavelengths: np.array of wavelengths
    :param bad_regions: list of lists containing the boundaries of the mask regions (default: [[1300,1500],[1800,2000]])
    :return: binary band mask (good bands are True, bad bands are False)
    """
    good_bands_mask = np.ones(wavelengths.shape, dtype=bool)
    for region in mask_regions:
        good_bands_mask = good_bands_mask * ~((wavelengths >= region[0]) & (wavelengths <= region[1]))
        
    return good_bands_mask


def prepare_image(fpath, 
                  mask_regions=[[300,416],[1300,1500],[1790,2000],[2400,2600]], 
                  scale_factor=10000, 
                  min_val=0, 
                  max_val=10000, 
                  no_data_pixels=-9999):
    """
    Prepare an image for MESMA.
    
    :param fpath: filepath to image
    :param mask regions: bad wavelength regions that should not be used for unmixing (default: wavelengths < 400 nm, water vapor bands at 1300-1500 nm and 1790-2000 nm, and wavelengths > 2400 nm)
    :param scale_factor: scale factor of reflectance data (MESMA requires values to be in range 0..1, default: 10000)
    :param min_val: lower boundary of accepted reflectance values, all values <= min_val will be set to no_data_pixels (default: 0)
    :param max_val: upper boundary of accepted reflectance values, all values > max_val will be set to no_data_pixels (default: 10000)
    :param no_data_pixels: value indicating no_data_pixels in MESMA (default: -9999)
    :return: 3D xarray.DataArray of shape (bands, spatial_x, spatial_y) with reflectance scales to range 0..1 and bands cropped to good wavelength regions
    """
    img = rioxarray.open_rasterio(fpath)
    img = img[band_mask(img.wavelength, mask_regions)] 
    return xr.where((img.min(dim='band')<=min_val) | (img.max(dim='band')>max_val), no_data_pixels*scale_factor, img).T / scale_factor   


def prepare_sli(fpath, mask_regions=[[300,416],[1300,1500],[1790,2000],[2400,2600]], scale_factor=1, num_bands=428):
    """
    Prepare a spectral library for MESMA.
    
    :param fpath: filepath to SLI
    :param mask regions: bad wavelength regions that should not be used for unmixing (default: wavelengths < 400 nm, water vapor bands at 1300-1500 nm and 1790-2000 nm, and wavelengths > 2400 nm)
    :param scale_factor: scale factor of reflectance data (MESMA requires values to be in range 0..1, default: 10000)
    :param num_bands: number of bands in SLI
    :return: two np.arrays:
        (1) 1D np.array of strings, a class for each endmember in the library 
        (2) 2D np.array of floats and shape (bands, endmembers) with reflectance scaled to range 0..1 and bands cropped to good wavelength regions
    """
    sli = pd.read_csv(fpath)
    class_list = sli.MaterialClass.values
    em_spectra = sli[sli.columns[-num_bands:][band_mask(sli.columns[-num_bands:].astype(float), mask_regions)]] / scale_factor
    
    return class_list, em_spectra.T


def save_to_disk(arr, source_filepath, output_filepath):
    """
    Save an array as a GeoTIFF using the profile of another raster.
    
    :param arr: np.array to save to disk
    :param source_filepath: filepath to file to use profile from
    :param output_filepath: filepath of new file based on profile of source_filepath containing array
    :return: None
    """
    dummy = rioxarray.open_rasterio(source_filepath)
    # read image profile (ENVI header data)
    profile = dummy.rio._manager.acquire().profile
    # update profile
    profile["dtype"] = 'float64'
    profile["driver"] = 'GTiff'
    profile['interleave']='pixel'
    profile["count"] = arr.shape[0]
    # use rasterio to write file to disk
    with rasterio.open(output_filepath, "w", **profile) as dst:
        dst.write(arr)

In [ ]:
folder_path = "path to folder that contains the image to unmix"
img_name = "name of image to unmix"

img = prepare_image(fpath=img_folder+img_name)
class_list, em_spectra = prepare_sli(fpath="path to spectral library (e.g., Tanager_SLI.csv) as provided in the GitHub repository")

In [ ]:
em_models = mesma.MesmaModels()
em_models.setup(class_list)

In [ ]:
# This code controls the class levels in MESMA. 
# Per default MESMA runs only 2-EM and 3-EM models. We use this code to add the 4-EM models.
# It is part of the function 'setup' in the class 'MesmaModels' (https://mesma.readthedocs.io/en/latest/_modules/mesma/core/mesma.html#MesmaModels.setup).

em_models.select_level(state=True, level=2)
em_models.select_level(state=False, level=3)
em_models.select_level(state=False, level=4)

for i in np.arange(em_models.n_classes):
    em_models.select_class(state=True, index=i, level=2)
    em_models.select_class(state=False, index=i, level=3)
    em_models.select_class(state=False, index=i, level=4)

In [ ]:
import timeit

# Arrays to fill with chunks during for-loop
out_models = np.zeros((len(np.unique(class_list)), img.shape[1], img.shape[2])) * np.nan 
out_fractions = np.zeros((len(np.unique(class_list)) + 1, img.shape[1], img.shape[2])) * np.nan # + 1 for the shade fraction
out_rmse = np.zeros((img.shape[1], img.shape[2])) * np.nan
out_residuals = np.zeros((img.shape)) * np.nan
# list to store all MESMA information for each chunk
out_dicts = []

total_start = timeit.default_timer()

start_row = 0
split_size = 10 # number of rows per chunk to be unmixed

for chunk in range(start_row, img.shape[1], split_size):
    
    start = timeit.default_timer()
    
    # Create MESMA object.
    MESMA = mesma.MesmaCore(n_cores=8) # number of cores for parallel processing; the look-up-table of endmember combinations is split into n parts used for parallel unmixing, several image copies are stored in memory during unmixing.

    # Execute MESMA.
    models, fractions, rmse, residuals = MESMA.execute(image = img[:, start_row:start_row+split_size, :].data, # image to be unmixed as np.array with shape (spatial_x, spatial_y, bands), needs to have same band setting as library and reflectance values ranging 0..1.
                                            library = em_spectra, # endmember spectra as 2D np.array (spectra as columns), needs to have same band setting as image and reflectance values ranging 0..1.               
                                            look_up_table = em_models.return_look_up_table(), # all endmember combinations (=models) for MESMA; ordered per complexity level and perclass-model; n_models x n_endmembers
                                            em_per_class = em_models.em_per_class, # a list of all library indices per endmember class
                                            constraints = (-0.05, 1.05, -0.1, 0.8, 0.05, -9999, -9999), # min + max endmember fraction, min + max shade fraction, max rmse, residual reflectance threshold + max number of consecutive bands exceeding threshold. set value to -9999 if not used, default: (-0.05, 1.05, 0., 0.8, 0.025, -9999, -9999)
                                            no_data_pixels = np.where(img[0, start_row:start_row+split_size, :].data==-9999), # indices of pixels that contain no data (result of np.where), -9999 for GAO data
                                            shade_spectrum = None, # single spectrum of photometric shade: default: None
                                            fusion_value = 0.01, # only select a model of higher complexity (e.g. 3-EM over 2-EM) of the RMSE is better with at least this value, default: 0.007
                                            residual_image = True, # output the residuals as an image (ignored when using band weighing or -selection), default: False
                                            use_band_weighing = False, # use the weighted linear spectral mixture analysis (Somers et al, 2009), default: False
                                            use_band_selection = False, # use the bands selection algorithm (Somers et al, 2010), default: False
                                            bands_selection_values = (0.99, 0.01) # correlation threshold and decrease for the band selection algorithm, default: (0.99, 0.01)
                                           )
    
    # Store chunk outputs in prepared arrays.
    out_models[:, start_row:start_row + split_size, :] = models
    out_fractions[:, start_row:start_row + split_size, :] = fractions
    out_rmse[start_row:start_row + split_size,:] = rmse
    out_residuals[:, start_row:start_row + split_size, :] = residuals
    out_dicts.append(MESMA.__dict__)
    
    start_row = start_row + split_size
    
    stop = timeit.default_timer()
    print('Chunk Time: ', stop - start)  
    
total_stop = timeit.default_timer()
print('Total Time: ', total_stop - total_start)  

In [ ]:
save_to_disk(out_models.transpose(0,2,1), 
            folder_path+img_name, 
            folder_path+img_name.split('.')[0]+"_models.tif")

save_to_disk(out_fractions.transpose(0,2,1), 
            folder_path+"filepath to file to use profile from", 
            folder_path+img_name.split('.')[0]+"_fractions.tif")

save_to_disk(out_residuals.transpose(0,2,1), 
            folder_path+"filepath to file to use profile from", 
            folder_path+img_name.split('.')[0]+"_residuals.tif")

save_to_disk(out_rmse[..., np.newaxis].transpose(2,1,0), 
            folder_path+"filepath to file to use profile from", 
            folder_path+img_name.split('.')[0]+"_rmse,tif")

save_to_disk(hard_classification.HardClassification().execute(out_fractions)[..., np.newaxis].transpose(2,1,0), 
            folder_path+"filepath to file to use profile from", 
            folder_path+img_name.split('.')[0]+"_classification.tif")

pd.DataFrame(out_dicts).to_csv(folder_path+img_path+img_name.split('.')[0]+"_params.csv")